In [ ]:
import os
import os.path

from collections import defaultdict

from h5Nastran import H5Nastran, pyNastranReadBdfError, pyNastranWriteBdfError

# this file verifies that h5Nastran can read all the bdf files included with pyNastran
# any bdf cards that are not supported are printed out


# path to pyNastran models, you will need to point to the correct path here
models_path = r'P:\redmond\pyNastran\models'

bdfs = []

for dirpath, dirnames, filenames in os.walk(models_path):
    for filename in [f for f in filenames if f.endswith('.bdf') or f.endswith('.dat')]:
        bdfs.append(os.path.join(dirpath, filename))

unsupported_cards = defaultdict(int)

for bdf in bdfs:
    db = H5Nastran('dummy.h5', 'w', in_memory=True)
    try:
        db.load_bdf(bdf)
    except (pyNastranReadBdfError, pyNastranWriteBdfError):
        db.close()
        continue

    for card_id in db._unsupported_bdf_cards:
        unsupported_cards[card_id] += 1

    db.close()

unsupported_cards = [(card_id, unsupported_cards[card_id]) for card_id in unsupported_cards.keys()]
unsupported_cards = sorted(unsupported_cards, key=lambda x: x[1])

for card in unsupported_cards:
    print(card)

print(len(unsupported_cards))

In [ ]:
from __future__ import print_function

import sys

# sys.path.insert(0, r'P:\redmond\pyNastran')

import numpy as np

from h5Nastran import H5Nastran

db = H5Nastran('./models/model_001.h5', 'w')  # , in_memory=True)
db.load_bdf('./models/model_001.bdf')
db.load_punch('./models/model_001.pch')
db.close()
db = H5Nastran('./models/model_001.h5', 'r')
db.load_bdf()

print(db.nastran.input.node.grid.identity)  # or db.input.node.grid.grid

domain_ids = range(10)
elements = [1, 5, 75]
f = db.nastran.result.elemental.element_force.quad4.search(elements, domain_ids)

print(f)

print(2 * f + 3 * f)

f = db.nastran.result.elemental.element_force.search(elements, domain_ids)

print(f.quad4)

# vec = db.nastran.input.coordinate_system.h5n_transformation.vector_to_basic([1, 1, 1], 1)

# print(vec)

# pynastran bdf
bdf = db.bdf

# currently, to modify the bdf and rewrite to h5,
# you'd need to modify the pynastran bdf, write it to a new file and create a new h5 database

# currently, the entire bdf is written to h5 as written by pynastran
# it can be loaded by doing db.load_bdf() without a filename
# the goal is to recreate the bdf file from only the h5 data

db.visualize()

db.close()
